In [1]:
import streamlit as st
from openai import OpenAI
import os
import re

In [2]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [5]:
JOB_DESCRIPTION = """AUFGABEN
                Entwicklung und Implementierung von Machine Learning-/KI-Modellen für Anwendungen, z.B. im Bereich Natural Language Processing (NLP)
                oder Computer Vision, mit Schwerpunkt auf OpenAI-Technologien (LLM).
                Enge Zusammenarbeit mit anderen Data Scientisten und unseren Geschäftsbereichen, Fachexperten*innen, internen und externen Dienstleistern.
                Fallbasierte Bewertung und Beratung zur Verwendung analytischer Methoden und Ansätze (insbesondere OpenAI-Technologien) zur Verbesserung unserer Prozesse und Dienstleistungen.
                Unterstützung von Geschäftsbereichen als interner Entwicklungspartner, von der Ideenfindung bis zur Implementierung.
                QUALIFIKATIONEN
                Abgeschlossenes Masterstudium in Informatik, Mathematik, Physik oder eine vergleichbare Qualifikation mit Schwerpunkt auf Maschine Learning / Künstliche Intelligenz.
                Drei Jahre Erfahrung in der Implementierung von Maschine Learning- / KI -Modellen, einschließlich der Verwendung von OpenAI-Technologien.
                Tiefgreifende Expertise über aktuelle Methoden im Bereich ML und KI, insbesondere Large Language Models.
                Erfahrung mit Cloud-Computing-Plattformen, speziell Azure. Umfassende Kenntnisse relevanter Programmiersprachen,
                einschließlich Python. Gute Kenntnisse in NLP und/oder Computer Vision. Starke analytische Fähigkeiten und die Fähigkeit, komplexe Probleme zu lösen.
                Ausgezeichnete schriftliche und mündliche Kommunikationsfähigkeiten in Deutsch und Englisch
                """

In [6]:
EVALUATION_PROMPT = f"""Job description: {JOB_DESCRIPTION}.
            As an AI interview assistant, your task is to evaluate the quality and depth of the candidate's responses and make a clear hiring decision for this job position.
            Consider the following:
            Does the candidate provide detailed answers that demonstrate their understanding and expertise?
            Can you find tangible examples in their responses that relate to the job description?
            Does the candidate elaborate on how they have used the necessary skills or experiences to overcome challenges or achieve results?
            Do the responses suggest the candidate has the ability to perform well in the role's complexities and challenges?
            If the candidate's responses are inadequate, vague, or don't clearly demonstrate the needed skills or experiences, they may not be a suitable match for the role. In such cases, tactfully communicate this by saying: "Thank you for your responses. However, based on the answers provided, it appears there may be a misalignment with the requirements of the role we're seeking to fill. At this time, we cannot extend an offer. We appreciate your time and effort and wish you the best in your future endeavors."
            If the responses indicate a strong fit for the role, then acknowledge the candidate's suitability by saying: "Thank you for your thoughtful responses. Based on your answers, it appears that your skills, experience, and understanding align well with the requirements of the role. We will be in touch with the next steps."
            """ 

In [7]:
def get_response_from_model(job_description=JOB_DESCRIPTION, nmb_of_questions=6):
    # Generate interview questions based on the job description
    prompt = f"Generate {nmb_of_questions} interview questions for a candidate applying for {job_description}."
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            *[{"role": "user", "content": ""} for i in range(nmb_of_questions)]
        ]
    )
    return response
 
def parse_model_response(response):  # TODO: check type openai.types.chat.chat_completion.ChatCompletion

    content = response.choices[0].message.content
    # questions = re.findall(f"\d.\s(.*)\\n*", content)
    questions = re.findall(r"\d\..*", content)
    print(questions)
    return [{question_id: question} for question_id, question in enumerate(questions)]
    

def conduct_interview(questions, answers=[]):
    # Conduct the interview
    print("Welcome to the interview!")
    print("Please answer the following questions with a maximum of 20 words each:")
    for i, question in enumerate(questions, 1):
        print(f"Question {i}: {question}")
        answer = input("Your answer: ")
        answers.append(answer)
        print()
    print("Thank you for your answers. We will now evaluate your suitability for the position.")
    return answers

def make_hiring_decision(answers):
    # Make the hiring decision
    prompt = EVALUATION_PROMPT
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": " ".join(answers)}
        ],
        presence_penalty=0.2,
        temperature=0.1
    )
    return response

In [10]:
response = get_response_from_model()
questions = parse_model_response(response)

In [2]:
# usesr_answers = conduct_interview(questions)

In [3]:
# make_hiring_decision(usesr_answers)